In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
# matchgap : 전경기와의 매치 차이 수
# month : 경기진행월
# matchtype : 0-리그, 1-챔스
# homeaway : 0-home, 1-away
# Team : 상대방팀
# TeamRanking : 상대방 팀랭킹
# PreScoreGoal : 전경기 득점수
# PreGetGoal : 전경기 실점수
# PrePreScoreGoal : 전전경기 득점수
# PrePreGetGoal : 전전경기 실점수
# RamosTime : 전경기 라모스 출장시간
# RamosGoal : 전경기 라모스 골
# RamosAss : 전경기 라모스 어시
# Results : 승,무,패

In [3]:
CSV_COLUMN_NAMES = ['MatchGap', 'Month', 'MatchType', 'HomeAway','Team', 'TeamRanking', 'PreScoreGoal', 'PrePreScoreGoal', 'PreGetGoal', 'PrePreGetGoal', 'RamosTime','RamosGoal', 'RamosAss','Results']
RESULTS = ['Win', 'Draw', 'Lose']

In [4]:
train = pd.read_csv('/content/real_training2.csv', names=CSV_COLUMN_NAMES)

In [6]:
train.head(5)

,MatchGap,Month,MatchType,HomeAway,Team,TeamRanking,PreScoreGoal,PrePreScoreGoal,PreGetGoal,PrePreGetGoal,RamosTime,RamosGoal,RamosAss,Results
0,5,9,1,1,60,12,0,2,3,3,90,0,0,2
1,4,9,1,0,8,7,0,0,1,3,68,0,0,0
2,3,9,1,1,25,20,3,0,1,1,0,0,0,0
3,3,9,0,0,53,0,3,3,0,1,90,0,0,0
4,4,10,1,0,61,19,2,3,1,0,90,0,0,0


In [7]:
train_y = train.pop('Results')

# The label column has now been removed from the features.
train.head()

,MatchGap,Month,MatchType,HomeAway,Team,TeamRanking,PreScoreGoal,PrePreScoreGoal,PreGetGoal,PrePreGetGoal,RamosTime,RamosGoal,RamosAss
0,5,9,1,1,60,12,0,2,3,3,90,0,0
1,4,9,1,0,8,7,0,0,1,3,68,0,0
2,3,9,1,1,25,20,3,0,1,1,0,0,0
3,3,9,0,0,53,0,3,3,0,1,90,0,0
4,4,10,1,0,61,19,2,3,1,0,90,0,0


In [8]:
# np.array가 어떤 의미인지도 모르고 그냥 의미없이 숫자를 집어넣었었다

def input_evaluation_set():
    features = {'MatchGap': np.array([3.0, 44.0]),
                'Month':  np.array([1.0, 12.0]),
                'MatchType': np.array([0.0, 2.0]),
                'HomeAway':  np.array([0.0, 1.0]),
                'Team':  np.array([0.0, 91.0]),
                'TeamRanking':  np.array([0.0, 21.0]),
                'PreScoreGoal':  np.array([0.0, 9.0]),
                'PrePreScoreGoal':  np.array([0.0, 9.0]),
                'PreGetGoal':  np.array([0.0, 9.0]),
                'PrePreGetGoal':  np.array([0.0, 6.0]),
                'RamosTime':  np.array([0.0, 120.0]),
                'RamosGoal':  np.array([0.0, 2.0]),
                'RamosAss':  np.array([0.0, 2.0]), }
    labels = np.array([0, 2])
    return features, labels

In [10]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

In [11]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [12]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[50, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfyuezsro', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=20000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpfyuezsro/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 5.349985, step = 0
INFO:tensorflow:global_step/sec: 240.485
INFO:tensorflow:loss = 1.459376, step = 100 (0.420 sec)
INFO:tensorflow:global_step/sec: 271.046
INFO:tensorflow:loss = 1.2888572, step = 200 (0.370 sec)
INFO:tensorflow:global_step/sec: 

# final loss : 0.78

In [19]:
# Generate predictions from the model
expected = ['Draw']
predict_x = {
    'MatchGap': [7], # 7일후 경기
    'Month': [2], #2월 경기
    'MatchType': [0], #리그경기
    'HomeAway': [1], #어웨이경기
    'Team': [71], #우에스카 71번팀
    'TeamRanking': [20], #우에스카 현재 20위
    'PreScoreGoal': [1], #레알 전경기 득점 1
    'PrePreScoreGoal': [4], #레알 전전경기 득점 4
    'PreGetGoal': [2], #레알 전경기 실점 2
    'PrePreGetGoal': [2], #레알 전전경기 실점 1
    'RamosTime': [00], #라모스 전경기 출전 0 # 부상인듯
    'RamosGoal': [0],
    'RamosAss': [0]}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

In [18]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        RESULTS[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfyuezsro/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Win" (48.0%), expected "Lose"
